<a href="https://colab.research.google.com/github/commd/08-2016-infinite-loops/blob/master/Selenium_Yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [0]:
!pip install selenium
!apt-get update -qq
!apt install chromium-chromedriver -qq

In [0]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


# Try out a few things

## Load the web page

In [0]:
url_page = 'https://www.yelp.com/biz/orchard-city-kitchen-campbell'
url_page = 'https://www.yelp.com/biz/puranpoli-santa-clara'

In [0]:
driver.get(url_page)

## Get one Comment, name, date and message

In [0]:
#getting how many messages are in this page
ids = driver.find_elements_by_xpath("//*[contains(@class,'lemon--div__373c0__1mboc user-passport-info border-color--default__373c0__3-ifU')]")
len(ids)

20

In [0]:
xpath = "//li[1]//div[1]//div[2]//div[1]//div[1]//div[1]//span[1]//div[1]"
userid_element = driver.find_elements_by_xpath(xpath)[0]
userid = userid_element.get_attribute("aria-label")
print(userid)

5 star rating


In [0]:
for i in range(0,len(ids)):
  xpath = "//li[" + str(i+1) + "]//div[1]//div[2]//div[1]//div[1]//div[1]//span[1]//div[1]"
  rating_element = driver.find_elements_by_xpath(xpath)[0]
  rating = rating_element.get_attribute("aria-label")
  print(rating)



5 star rating
5 star rating
5 star rating
5 star rating
5 star rating
4 star rating
4 star rating
4 star rating
3 star rating
5 star rating
5 star rating
5 star rating
3 star rating
5 star rating
4 star rating
4 star rating
4 star rating
5 star rating
3 star rating
3 star rating


In [0]:
import pandas as pd


In [0]:
url_page = 'https://www.yelp.com/biz/orchard-city-kitchen-campbell'
#url_page = 'https://www.yelp.com/biz/puranpoli-santa-clara'
page_num = 0
cnt = 0
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.get(url_page)
entity_name = url_page.split("/")[-1]

user_message = driver.find_elements_by_xpath("//span[contains(text(),'1 of ')]")[0]
comment = user_message.text.split()
pages = int(comment[2])

msg_pd =  pd.DataFrame(columns = ['Entity','UserName',"Date", "Rating",'Comment']) 


while (page_num < min(pages,20)):
  ids = driver.find_elements_by_xpath("//*[contains(@class,'lemon--div__373c0__1mboc user-passport-info border-color--default__373c0__3-ifU')]")
  print(page_num)
  for i in range(0,len(ids)):
    name_path =  "//div[@id='wrap']/div/div/div/div/div/div/div/div/section/div/div/ul/li[" + str(i+1) + "]/div[1]"
    name_element = driver.find_elements_by_xpath(name_path)[0]
    name = name_element.text.split("\n")

    msg_date = driver.find_elements_by_xpath("//div[@id='wrap']/div/div/div/div/div/div/div/div/section/div/div/ul/li[" + str(i+1) +  "]/div[1]/div[2]")[0]
    date_text = msg_date.text.split("\n")[0]

    xpath = "//li[" + str(i+1) + "]//div[1]//div[2]//div[1]//div[1]//div[1]//span[1]//div[1]"
    rating_element = driver.find_elements_by_xpath(xpath)[0]
    rating = rating_element.get_attribute("aria-label")

    name_path =  "//div[@id='wrap']/div/div/div/div/div/div/div/div/section/div/div/ul/li[" + str(i+1) + "]/div[1]/div[2]/div[2]"
    userid_element = driver.find_elements_by_xpath(name_path)[0]
    msg = userid_element.text
    words = msg.split()
    if len(words) > 1:
      last_word = words[1]
      if last_word == "photos" or last_word == "photo":
        msg_path = "//div[@id='wrap']/div/div/div/div/div/div/div/div/section/div/div/ul/li[" + str(i+1) + "]/div[1]/div[2]/div[3]"
        msg_element = driver.find_elements_by_xpath(msg_path)[0]
        msg = msg_element.text

    #print("===>", name[0], "===>",msg)
    msg_pd.loc[len(msg_pd)] = [entity_name, name[0],date_text,rating, msg]
  cnt += len(ids)
  print(page_num)
  page_num += 1
  driver.get(url_page + "?start=" + (str(cnt)))
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
CPU times: user 5.92 s, sys: 234 ms, total: 6.16 s
Wall time: 3min 58s


In [0]:
msg_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Entity    400 non-null    object
 1   UserName  400 non-null    object
 2   Date      400 non-null    object
 3   Rating    400 non-null    object
 4   Comment   400 non-null    object
dtypes: object(5)
memory usage: 18.8+ KB


In [0]:
msg_pd.head()

,Entity,UserID,Date,Rating,Comment
0,puranpoli-santa-clara,Cara R.,5/9/2020,5 star rating,This place is a gem -- they opened at such an ...
1,puranpoli-santa-clara,M V.,5/8/2020,5 star rating,We ordered food from here (during the covid cr...
2,puranpoli-santa-clara,Akhilesh B.,3/20/2020,5 star rating,Awesome Maharashtrian food! Brings back memori...
3,puranpoli-santa-clara,Abdul Q.,5/14/2020,4 star rating,"This is a nice place with nice people, claimin..."
4,puranpoli-santa-clara,Mahima S.,5/8/2020,5 star rating,


In [0]:
msg_pd.tail()

,UserID,Date,Comment
74,Swathi N.,4/16/2020,"Amazing food, friendly staff, best place in sa..."
75,Kooshiar V.,4/6/2020,"Great new spot, found them through Instagram. ..."
76,A G.,4/17/2020,Good food at affordable prices. I really liked...
77,Kasturi P.,4/11/2020,"Amazing Maharashtrian food! Must try, hatsoff ..."
78,Manish A.,3/21/2020,I was born and brought up in Maharashtra. This...


# Write to Google Sheets

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# gspread will pull new version of auth which is not compatible with colab V1.0.0
!pip install --upgrade -q gspread

In [0]:
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
import gspread_dataframe as gd

# rememmber to create a google spreadsheet with sheet name reviews.
ws = gc.open('Yelp Reviews').worksheet("Reviews")
existing = gd.get_as_dataframe(ws)
updated = existing.append(msg_pd)
gd.set_with_dataframe(ws, updated)
